In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
img_rgb = cv2.imread('test1.png')
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
template = cv2.imread('4_masked.jpg', 0)

height, width = template.shape[::]

res = cv2.matchTemplate(img_gray, template, cv2.TM_SQDIFF)
plt.imshow(res, cmap='gray')

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

top_left = min_loc  #Change to max_loc for all except for TM_SQDIFF
bottom_right = (top_left[0] + width, top_left[1] + height)
cv2.rectangle(img_rgb, top_left, bottom_right, (255, 0, 0), 2) 

cv2.imshow("Matched image", img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
img_path = "palm.png"

img = cv2.imread(img_path)
# plt.imshow(img)
# plt.show()
# cv2.imshow('palm image',img)
# cv2.waitKey()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [ ]:
hsvim = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")
skinRegionHSV = cv2.inRange(hsvim, lower, upper)
blurred = cv2.blur(skinRegionHSV, (2,2))
ret,thresh = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY)
# cv2.imshow("thresh", thresh)
# cv2.waitKey()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

In [ ]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = max(contours, key=lambda x: cv2.contourArea(x))
cv2.drawContours(img, [contours], -1, (255,255,0), 2)

# plt.imshow(img.astype('uint8'))
# plt.show()
cv2.imshow("contours", img)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
# ----------------
# a) Reading a stream of images from a webcamera, and displaying the video
# ----------------
# For more information on reading and writing video: http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html
# open the video camera no. 0
cap = cv2.VideoCapture(0)

# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

# create a window called "MyVideo0"
cv2.namedWindow("MyVideo0", cv2.WINDOW_AUTOSIZE)

# read a new frame from video
ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

# show the frame in "MyVideo" window
cv2.imshow("MyVideo0", frame0)

# create windows
cv2.namedWindow("Skin", cv2.WINDOW_AUTOSIZE)

len_history = 7
my_motion_history = []
fMH1 = np.zeros(np.shape(frame0)[:-1], dtype=np.uint8)
for i in range(len_history):
    my_motion_history.append(fMH1)


while(1):
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break

    cv2.imshow("MyVideo0", frame)

    # ----------------
    # b) Skin color detection
    # ----------------
    frame_dst = my_skin_detect(frame)
    cv2.imshow("Skin", frame_dst)

    # ----------------
    # c) Background differencing and motion energy
    # ----------------

    # call my_frame_differencing function
    frame_dst = my_frame_differencing(frame0, frame)
    cv2.imshow("FrameDiff", frame_dst)

    my_motion_history.pop(0)
    my_motion_history.append(frame_dst)

    # ----------------
    #  d) Visualizing motion history
    # ----------------

    # call my_motion_energy function
    myMH = my_motion_energy(my_motion_history)
    cv2.imshow("MotionEnergy", myMH) # show the frame in "MyVideo" window
    frame0 = frame

    # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()